In [1]:
using LinearAlgebra
using Optim
using HDF5

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [2]:
function infidelity_norm(ρ, σ)
    real(1 - tr(ρ * σ) - 2*sqrt(abs(det(ρ)*det(σ))))^2
end

infidelity_norm (generic function with 1 method)

In [3]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [4]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

initial_params .+= 1;

In [5]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [6]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"
γᶠ = parse(ComplexF64, γᵢ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γᵢ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γᵢ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γᵢ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γᵢ);

In [7]:
# Perform the optimization
result = optimize(objective, initial_params, 
                  method = BFGS(),
                  g_tol = 1e-8, show_trace = true;
                  autodiff = :forward)

Iter     Function value   Gradient norm 
     0     5.818044e+00     4.323898e+00
 * time: 0.021100997924804688
     1     4.598337e+00     3.523419e+00
 * time: 5.750668048858643
     2     4.492603e-01     6.397245e-02
 * time: 12.575934886932373
     3     2.843409e-01     4.881239e-02
 * time: 23.93993091583252
     4     2.702649e-01     4.554228e-02
 * time: 30.730376958847046
     5     2.632412e-01     4.347586e-02
 * time: 37.54203200340271
     6     1.870016e-01     3.232874e-02
 * time: 46.58540391921997
     7     1.032453e-01     3.170705e-02
 * time: 53.35028696060181
     8     8.005024e-02     5.051549e-02
 * time: 60.11813998222351
     9     5.923376e-02     2.043550e-02
 * time: 66.87504005432129
    10     3.807406e-02     9.614680e-03
 * time: 75.86381506919861
    11     3.457409e-02     1.181537e-02
 * time: 82.58694696426392
    12     2.690268e-02     1.134740e-02
 * time: 91.53958201408386
    13     2.358853e-02     1.257979e-02
 * time: 95.99976396560669
  

 * Status: success

 * Candidate solution
    Final objective value:     6.458945e-09

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.37e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 5.45e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 6.50e-10 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.01e-01 ≰ 0.0e+00
    |g(x)|                 = 5.50e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   485  (vs limit Inf)
    Iterations:    65
    f(x) calls:    225
    ∇f(x) calls:   225


In [8]:
# Extract optimized parameters
optimized_params = result.minimizer

println("Optimized parameters: ", optimized_params)

Optimized parameters: [25.16547239109983, -0.010886234376236745, 0.010300490326129452, 0.09907846058844363, -0.24710411060971088, 0.004979866064682022, 0.09907846058567774, -0.24710411060649748, 0.004979866068670096, 0.09907846058302104, -0.24710411060770895, 0.004979866069220867, -0.3123915266888333, -0.06651132033677061, 0.017802047778982838, -0.31239152668948095, -0.066511320338368, 0.017802047788239954, -0.31239152668555403, -0.06651132033874849, 0.017802047793108698]


In [9]:
# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

Optimized H: ComplexF64[12.582736195549915 + 0.0im -0.005443117188118372 + 0.005150245163064726im; -0.005443117188118372 - 0.005150245163064726im -12.582736195549915 + 0.0im]
Optimized C: ComplexF64[0.3222150218954007 + 0.0im -0.01111536594309653 + 0.25134920878290806im -0.01520343426716977 - 0.009958402360568021im; -0.01111536594309653 - 0.25134920878290806im 0.1964525916373684 + 0.0im -0.007243749235741411 + 0.012203225154284805im; -0.01520343426716977 + 0.009958402360568021im -0.007243749235741411 - 0.012203225154284805im 0.0010251359143595338 + 0.0im]


In [10]:
optimized_H

2×2 Matrix{ComplexF64}:
     12.5827+0.0im         -0.00544312+0.00515025im
 -0.00544312-0.00515025im     -12.5827+0.0im

In [11]:
optimized_C

3×3 Matrix{ComplexF64}:
   0.322215+0.0im         -0.0111154+0.251349im    -0.0152034-0.0099584im
 -0.0111154-0.251349im      0.196453+0.0im        -0.00724375+0.0122032im
 -0.0152034+0.0099584im  -0.00724375-0.0122032im   0.00102514+0.0im

In [12]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end



get_lindblad_operators (generic function with 1 method)

In [13]:
Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
Cˢⁱᵈ = convert.(ComplexF64,optimized_C)

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

1-element Vector{Any}:
 ComplexF64[0.01600887187124326 + 0.0im -0.4279917354870767 - 0.2686346629335212im; -0.04685228817207046 - 0.042392948526606425im -0.01600887187124326 + 0.0im]

In [14]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [15]:
using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
│ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.
└ @ nothing nothing:984


In [16]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9962790120313414
Median fidelity for 0.25133: 0.9981761124723088
